In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/home/anthony/summer-bursary-2018/bacteria'

In [3]:
os.listdir('data')

['Classific.py',
 'graphs.pdf',
 'growth_stages.csv',
 '16ms_32ms_growth_phase_spectra.csv',
 'PC6allwavelengths.png',
 'wavelengths.csv',
 '16_ms_lag_codes.csv']

In [4]:
df = pd.read_csv('data/growth_stages.csv', header=[0, 1, 2], index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Columns: 135 entries, (lag, bc, 01) to (stat, se, 07)
dtypes: float64(135)
memory usage: 1.1 MB


# Preprocessing

In [5]:
X = df.reorder_levels([1, 0, 2], axis=1).sort_index(axis=1).T
X.head()

0         1         2         3     \
species growth_stage replicate                                           
bc      lag          01         0.000000  1.560060  0.715025 -0.520060   
                     02         0.622353 -2.115990  0.062234  2.178335   
                     03         0.064528  0.645279 -1.032450 -2.064991   
                     04        -1.094650 -2.554170 -0.912210  0.851433   
                     05        -1.492810 -0.559800  2.923387  3.047967   

                                    4         5             6         7     \
species growth_stage replicate                                               
bc      lag          01         0.130014 -2.080180  0.000000e+00  1.040112   
                     02         1.867163  1.120305  8.713430e-01 -1.369260   
                     03        -1.419711  1.032515 -2.000491e+00 -0.903453   
                     04         3.709871  1.520432  8.514440e-01 -2.067810   
                     05         2.985793  2.985763 -3.590000e-07 -0.995270   

                                    8         9       ...         1033  \
species growth_stage replicate                        ...                
bc      lag          01         1.235136 -0.195020    ...     1.170374   
                     02        -1.929420  0.684622    ...    -0.373450   
                     03         2.968462  1.032499    ...     3.097847   
                     04        -1.581270  0.304085    ...     4.257782   
                     05        -0.933060 -1.990480    ...     4.479034   

                                    1034      1035      1036      1037  \
species growth_stage replicate                                           
bc      lag          01         2.730861  3.835922  1.755222  1.235332   
                     02        -0.187040 -2.302970  0.124451  0.809098   
                     03         4.775991  4.775919  4.259427  2.129823   
                     04         3.284733  0.851676  2.493739  3.467134   
                     05         2.488391  3.546049  1.057399  3.981489   

                                    1038      1039      1040      1041  \
species growth_stage replicate                                           
bc      lag          01         4.355984  0.520024  0.715032 -0.325010   
                     02         1.058059  2.676111  0.622357  0.560118   
                     03         0.064477 -2.516599  0.516229 -0.903391   
                     04         5.900169  1.520354  2.067671  0.121629   
                     05         6.469838 -0.933000  0.062199 -0.435400   

                                    1042  
species growth_stage replicate            
bc      lag          01        -0.520030  
                     02        -0.435650  
                     03         1.096991  
                     04         1.094670  
                     05         1.430629  

[5 rows x 1043 columns]

In [6]:
y = X.reset_index()['species']
y.head()

0    bc
1    bc
2    bc
3    bc
4    bc
Name: species, dtype: object

In [7]:
y.describe()

count     135
unique      6
top        bc
freq       36
Name: species, dtype: object

In [8]:
print(y)

0      bc
1      bc
2      bc
3      bc
4      bc
5      bc
6      bc
7      bc
8      bc
9      bc
10     bc
11     bc
12     bc
13     bc
14     bc
15     bc
16     bc
17     bc
18     bc
19     bc
20     bc
21     bc
22     bc
23     bc
24     bc
25     bc
26     bc
27     bc
28     bc
29     bc
       ..
105    sa
106    sa
107    sa
108    sa
109    sa
110    sa
111    sa
112    sa
113    sa
114    sa
115    sa
116    sa
117    sa
118    se
119    se
120    se
121    se
122    se
123    se
124    se
125    se
126    se
127    se
128    se
129    se
130    se
131    se
132    se
133    se
134    se
Name: species, Length: 135, dtype: object


In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(X_scaled.mean(), X_scaled.std())
print(X_scaled.shape)

2.9066625176507776e-17 0.9999999999999999
(135, 1043)


In [11]:
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y.values.reshape(-1, 1))

print(y_ohe.shape)
print(y_ohe[:5])

(135, 6)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


# PCA + SVM

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X_scaled)

X_pca = pca.transform(X_scaled)
print(X_pca.shape)

(135, 74)


In [13]:
def gaussian_kernel(X, Y):
    raise NotImplementedError

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV

param_grid = {
    'gamma': [10 ** -n for n in range(11)],
    'C': [10 ** n for n in range(-10, 2)]
}

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
clf = SVC()

grid_search = GridSearchCV(clf, param_grid, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 60 folds for each of 132 candidates, totalling 7920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 2505 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 4605 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done 7305 tasks      | elapsed:  1.2min


0.43222222222222223
{'C': 1, 'gamma': 1e-09}


[Parallel(n_jobs=-1)]: Done 7920 out of 7920 | elapsed:  1.3min finished
/home/anthony/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [15]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=100)
scores = cross_val_score(grid_search.best_estimator_, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.27 (+/- 0.02)
CPU times: user 1.08 s, sys: 0 ns, total: 1.08 s
Wall time: 1.08 s


# Random Forest with Decision Stumps

In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.33 (+/- 0.06)
CPU times: user 47.8 s, sys: 22 ms, total: 47.8 s
Wall time: 47.8 s


# AdaBoost

In [17]:
%%time

from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(RandomForestClassifier(n_estimators=1024, max_depth=1),
                        n_estimators=512)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=10)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.37 (+/- 0.10)
CPU times: user 4min 16s, sys: 188 ms, total: 4min 16s
Wall time: 4min 16s
